## Signal Chain in QCoDeS

The purpose of the signal chain is to eventually make it possible to account for modifications made to the instrument output after the signal leaves the instrument, e.g. attenuation along the signal line.

This notebook shows how to add a signal chain to a StandardParameter of a dummy instrument.

In [1]:
import qcodes as qc
from qcodes.instrument.parameter import StandardParameter
from qcodes.instrument.signalchain import addToSignalChain, removeFromSignalChain

In [2]:
from qcodes.tests.instrument_mocks import DummyInstrument
import numpy as np


In [3]:
# Set up two dummy instruments and give them a parameter each
dac = DummyInstrument(name='dac')
dmm = DummyInstrument(name='dmm')

dmm.add_parameter('voltage',
                  set_cmd=lambda x: x,
                  get_cmd=lambda : np.random.randn(),  # random return values for good looks
                  unit='V')


dac.add_parameter('voltage',
                  set_cmd=lambda x: x,
                  unit='V')

In [4]:
# A loop of the 'dac' voltage whilst reading from the 'dmm' will return noise centred on zero with an std of 1
loop = qc.Loop(dac.voltage.sweep(0, 1, 0.05), delay=0.01).each(dmm.voltage)
data = loop.get_data_set(name='testsweep')
plot = qc.QtPlot()
plot.add(data.dmm_voltage)
_ = loop.with_bg_task(plot.update, plot.save).run()

DataSet:
   location = 'data/2017-04-07/#005_testsweep_10-40-42'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | dac_voltage_set | voltage      | (21,)
   Measured | dmm_voltage     | voltage      | (21,)
started at 2017-04-07 10:40:45


### Add to signal chain

We now add an offset, a multiplier, and a second offset.

In [5]:
addToSignalChain(dmm.voltage, 'offset', 48)
addToSignalChain(dmm.voltage, 'multiplier', 1e-3)
addToSignalChain(dmm.voltage, 'offset', 1)

In [6]:
# we can see the signal chain in the 'signalchain' attribute
dmm.voltage.signalchain

[('offset', 48), ('multiplier', 0.001), ('offset', 1)]

In [7]:
# We can query the raw (instrument) value by using raw_get

print('Value at target: {} ({})'.format(dmm.voltage(), dmm.voltage.unit))
print('Value at instrument: {} ({})'.format(dmm.voltage.raw_get(), dmm.voltage.unit))

# NB: in this example, the random number generator makes these two numbers tricky to compare directly
# just execute this cell a few times and convince yourself that this is sane

Value at target: 1.0484140107598805 (V)
Value at instrument: -0.284132921872809 (V)


In [8]:
# Performing the same loop again will return noise centred on 1.048 with an std of 1e-3
loop = qc.Loop(dac.voltage.sweep(0, 1, 0.05), delay=0.01).each(dmm.voltage)
data = loop.get_data_set(name='testsweep')
plot = qc.QtPlot()
plot.add(data.dmm_voltage)
_ = loop.with_bg_task(plot.update, plot.save).run()

DataSet:
   location = 'data/2017-04-07/#006_testsweep_10-40-57'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | dac_voltage_set | voltage      | (21,)
   Measured | dmm_voltage     | voltage      | (21,)
started at 2017-04-07 10:40:59


## Metadata

In the `snapshot.json` metadata, the `dmm.voltage` parameter has the following entry:

```
        "dmm_voltage": {
            "__class__": "qcodes.data.data_array.DataArray",
            "action_indices": [
                0
            ],
            "array_id": "dmm_voltage",
            "instrument": "qcodes.tests.instrument_mocks.DummyInstrument",
            "instrument_name": "dmm",
            "is_setpoint": false,
            "label": "voltage",
            "name": "voltage",
            "shape": [
                21
            ],
            "signalchain": [
                [
                    "offset",
                    48
                ],
                [
                    "multiplier",
                    0.001
                ],
                [
                    "offset",
                    1
                ]
            ],
            "unit": "V",
            "vals": "<Numbers>"
        }
```

### Remove from signal chain

Elements can be removed from the signal chain by using `removeFromSignalChain`. 

In [9]:
dmm.voltage.signalchain

[('offset', 48), ('multiplier', 0.001), ('offset', 1)]

In [10]:
# Enter the 1-indexed number of the element to remove, counting from the instrument.
# In our case, that means

# removeFromSignalChain(dmm.voltage, 1)  # remove the offset of 48 
# removeFromSignalChain(dmm.voltage, 2)  # remover the multiplier of 0.001
removeFromSignalChain(dmm.voltage, 3)  # remove the offset of 1

In [11]:
dmm.voltage.signalchain

[('offset', 48), ('multiplier', 0.001)]

In [12]:
dmm.voltage()

0.04904794618130652